# 04 - K-Means Anomaly Detection

## Method 2 of 4: Clustering Approach

### What is K-Means?

K-Means groups similar data points into **clusters**.

**How it works:**
1. Choose K (number of clusters)
2. Place K random center points
3. Assign each data point to nearest center
4. Move centers to middle of their points
5. Repeat until stable

### How to Detect Anomalies

**Key insight:** Anomalies are FAR from any cluster center.

```
    Normal points          Anomaly
    (near center)         (far from all)
         •  •                  
        • ○ •                   ×
         • •                  
```

### Pros & Cons

| Pros | Cons |
|------|------|
| Considers ALL features together | Must choose K |
| Fast and scalable | Assumes spherical clusters |
| Works in Spark MLlib | Sensitive to outliers |

---
## Step 1: Setup
---

In [1]:
# ═══════════════════════════════════════════════════════════════════
# IMPORTS
# ═══════════════════════════════════════════════════════════════════

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, mean, stddev, when, lit, udf, monotonically_increasing_id
from pyspark.sql.types import DoubleType
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.clustering import KMeans
import numpy as np
import os

print("✓ Libraries imported")

✓ Libraries imported


In [2]:
# ═══════════════════════════════════════════════════════════════════
# CREATE SPARK SESSION
# ═══════════════════════════════════════════════════════════════════

spark = SparkSession.builder \
    .appName("KMeans_AnomalyDetection") \
    .master("local[*]") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

print("✓ Spark Session created")
print(f"  Version: {spark.version}")

✓ Spark Session created
  Version: 3.5.0


In [3]:
# ═══════════════════════════════════════════════════════════════════
# CONFIGURATION
# ═══════════════════════════════════════════════════════════════════

# File paths
DATA_PATH = "../data/processed/BTCUSDT_1h_processed.csv"
OUTPUT_PATH = "../data/results/"

# Feature columns
FEATURE_COLUMNS = [
    "return",
    "log_return",
    "volatility_24h",
    "volume_change",
    "volume_ratio",
    "price_range"
]

# K-Means parameters
K_CLUSTERS = 3  # Number of clusters

print("✓ Configuration set")
print(f"  Clusters: {K_CLUSTERS}")

✓ Configuration set
  Clusters: 3


---
## Step 2: Load and Prepare Data
---

In [4]:
# ═══════════════════════════════════════════════════════════════════
# LOAD DATA
# ═══════════════════════════════════════════════════════════════════

df = spark.read.csv(DATA_PATH, header=True, inferSchema=True)

print("✓ Data loaded")
print(f"  Rows: {df.count()}")

✓ Data loaded
  Rows: 976


In [5]:
# ═══════════════════════════════════════════════════════════════════
# PREPARE FEATURES: VectorAssembler + StandardScaler
# ═══════════════════════════════════════════════════════════════════

# Step 1: Combine features into vector
assembler = VectorAssembler(
    inputCols=FEATURE_COLUMNS,
    outputCol="features_raw"
)
df_assembled = assembler.transform(df)

# Step 2: Scale features (mean=0, std=1)
scaler = StandardScaler(
    inputCol="features_raw",
    outputCol="features",
    withMean=True,
    withStd=True
)
scaler_model = scaler.fit(df_assembled)
df_scaled = scaler_model.transform(df_assembled)

print("✓ Features prepared")
print("  • VectorAssembler: Combined 6 features into vector")
print("  • StandardScaler: Scaled to mean=0, std=1")

✓ Features prepared
  • VectorAssembler: Combined 6 features into vector
  • StandardScaler: Scaled to mean=0, std=1


---
## Step 3: Train K-Means Model
---

In [6]:
# ═══════════════════════════════════════════════════════════════════
# TRAIN K-MEANS MODEL
# ═══════════════════════════════════════════════════════════════════

kmeans = KMeans(
    k=K_CLUSTERS,
    featuresCol="features",
    predictionCol="cluster",
    seed=42
)

kmeans_model = kmeans.fit(df_scaled)

print("✓ K-Means model trained")
print(f"  Clusters: {K_CLUSTERS}")

✓ K-Means model trained
  Clusters: 3


In [7]:
# Show cluster centers
print("Cluster Centers (scaled feature space):")
print("-" * 60)
centers = kmeans_model.clusterCenters()
for i, center in enumerate(centers):
    print(f"Cluster {i}: {[f'{x:.2f}' for x in center]}")

Cluster Centers (scaled feature space):
------------------------------------------------------------
Cluster 0: ['-0.04', '-0.04', '-0.06', '-0.19', '-0.28', '-0.32']
Cluster 1: ['-2.25', '-2.26', '0.50', '1.25', '1.79', '2.21']
Cluster 2: ['1.64', '1.63', '0.25', '0.95', '1.40', '1.49']


In [8]:
# ═══════════════════════════════════════════════════════════════════
# ASSIGN POINTS TO CLUSTERS
# ═══════════════════════════════════════════════════════════════════

df_clustered = kmeans_model.transform(df_scaled)

print("Cluster distribution:")
df_clustered.groupBy("cluster").count().orderBy("cluster").show()

Cluster distribution:
+-------+-----+
|cluster|count|
+-------+-----+
|      0|  827|
|      1|   55|
|      2|   94|
+-------+-----+



---
## Step 4: Calculate Distance to Cluster Center
---

In [9]:
# ═══════════════════════════════════════════════════════════════════
# CALCULATE DISTANCE TO CLUSTER CENTER
# ═══════════════════════════════════════════════════════════════════

# Broadcast cluster centers for distributed computation
centers_broadcast = spark.sparkContext.broadcast(centers)

# UDF to calculate Euclidean distance
def distance_to_center(features, cluster):
    """Calculate Euclidean distance from point to its cluster center"""
    center = centers_broadcast.value[cluster]
    diff = np.array(features) - np.array(center)
    return float(np.sqrt(np.sum(diff ** 2)))

distance_udf = udf(distance_to_center, DoubleType())

# Add distance column
df_clustered = df_clustered.withColumn(
    "distance",
    distance_udf(col("features"), col("cluster"))
)

print("✓ Distance calculated")
print("\nDistance distribution:")
df_clustered.select("distance").describe().show()

✓ Distance calculated

Distance distribution:
+-------+-------------------+
|summary|           distance|
+-------+-------------------+
|  count|                976|
|   mean| 1.6190169011920477|
| stddev| 1.0225934517820672|
|    min|0.18476895384681388|
|    max| 13.256936443001278|
+-------+-------------------+



---
## Step 5: Flag Anomalies
---

In [10]:
# ═══════════════════════════════════════════════════════════════════
# DEFINE THRESHOLD: mean + 2*stddev
# ═══════════════════════════════════════════════════════════════════

distance_stats = df_clustered.agg(
    mean(col("distance")).alias("mean_dist"),
    stddev(col("distance")).alias("std_dist")
).first()

THRESHOLD = distance_stats["mean_dist"] + 2 * distance_stats["std_dist"]

print(f"Distance mean: {distance_stats['mean_dist']:.4f}")
print(f"Distance std:  {distance_stats['std_dist']:.4f}")
print(f"Threshold:     {THRESHOLD:.4f}")
print(f"\nPoints with distance > {THRESHOLD:.4f} are anomalies")

Distance mean: 1.6190
Distance std:  1.0226
Threshold:     3.6642

Points with distance > 3.6642 are anomalies


In [11]:
# ═══════════════════════════════════════════════════════════════════
# FLAG ANOMALIES
# ═══════════════════════════════════════════════════════════════════

df_clustered = df_clustered.withColumn(
    "is_anomaly",
    when(col("distance") > THRESHOLD, 1).otherwise(0)
)

# Count results
total = df_clustered.count()
anomalies = df_clustered.filter(col("is_anomaly") == 1).count()
normal = total - anomalies

print("\n" + "=" * 50)
print("K-MEANS RESULTS")
print("=" * 50)
print(f"Total data points:  {total}")
print(f"Normal:             {normal} ({100*normal/total:.1f}%)")
print(f"Anomalies:          {anomalies} ({100*anomalies/total:.1f}%)")
print("=" * 50)


K-MEANS RESULTS
Total data points:  976
Normal:             946 (96.9%)
Anomalies:          30 (3.1%)


---
## Step 6: Examine Anomalies
---

In [12]:
# ═══════════════════════════════════════════════════════════════════
# VIEW DETECTED ANOMALIES
# ═══════════════════════════════════════════════════════════════════

print("Sample anomalies detected (sorted by distance):")
print("-" * 60)

df_clustered.filter(col("is_anomaly") == 1) \
    .select("timestamp", "close", "return", "volume_change", "cluster", "distance") \
    .orderBy(col("distance").desc()) \
    .show(10, truncate=False)

Sample anomalies detected (sorted by distance):
------------------------------------------------------------
+-------------------+--------+-------------------+------------------+-------+------------------+
|timestamp          |close   |return             |volume_change     |cluster|distance          |
+-------------------+--------+-------------------+------------------+-------+------------------+
|2025-12-26 02:00:00|89199.99|2.021828098690359  |1113.137577476698 |2      |13.256936443001278|
|2026-01-21 16:00:00|87602.4 |-2.97452464357405  |93.2422892494962  |1      |9.55552286358597  |
|2025-12-18 13:00:00|88851.7 |1.8241005986821657 |611.2129031317518 |2      |8.165375376805965 |
|2026-01-19 00:00:00|92711.02|-1.0271033937796825|118.65475569224829|1      |7.469238009654748 |
|2026-01-18 23:00:00|93673.14|-1.9115147330416637|503.44753709511343|1      |7.401525051238573 |
|2026-01-21 19:00:00|90355.56|2.1134891925145283 |140.496295052039  |2      |7.224497912873434 |
|2026-01-13 22:00:

---
## Step 7: Save Results
---

In [13]:
# ═══════════════════════════════════════════════════════════════════
# SAVE RESULTS
# ═══════════════════════════════════════════════════════════════════

os.makedirs(OUTPUT_PATH, exist_ok=True)

# Prepare result DataFrame
df_result = df_clustered.withColumn("row_id", monotonically_increasing_id())
df_result = df_result.select(
    "row_id", "timestamp", "close", "return", "volume_change",
    "cluster", "distance",
    col("is_anomaly").alias("anomaly_kmeans")
)

# Save
output_file = OUTPUT_PATH + "kmeans_results"
df_result.coalesce(1).write.mode("overwrite").option("header", "true").csv(output_file)

print(f"✓ Results saved to: {output_file}")

✓ Results saved to: ../data/results/kmeans_results


In [14]:
# ═══════════════════════════════════════════════════════════════════
# STOP SPARK
# ═══════════════════════════════════════════════════════════════════

spark.stop()
print("✓ Spark stopped")
print("\n" + "=" * 50)
print("K-MEANS COMPLETE - Proceed to 05_random_forest.ipynb")
print("=" * 50)

✓ Spark stopped

K-MEANS COMPLETE - Proceed to 05_random_forest.ipynb
